In [194]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

In [195]:
dataset = pd.read_csv("../data/credito4.csv", sep=";")

dataset.head()

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,CLASSE
0,1169.0,4,67,nenhum,01/01/2019,masculino solteiro,radio/tv,bom
1,5951.0,2,22,nenhum,01/01/2020,fem div/cas,radio/tv,ruim
2,2096.0,3,49,nenhum,02/01/2020,masculino solteiro,educação,bom
3,7882.0,4,45,nenhum,02/01/2019,masculino solteiro,mobilia/equipamento,bom
4,4870.0,4,53,nenhum,03/01/2018,masculino solteiro,carro novo,ruim


In [196]:
dataset.shape

(1000, 8)

In [197]:
y = dataset["CLASSE"]

X = dataset.iloc[:, 0:7]

X.head()

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO
0,1169.0,4,67,nenhum,01/01/2019,masculino solteiro,radio/tv
1,5951.0,2,22,nenhum,01/01/2020,fem div/cas,radio/tv
2,2096.0,3,49,nenhum,02/01/2020,masculino solteiro,educação
3,7882.0,4,45,nenhum,02/01/2019,masculino solteiro,mobilia/equipamento
4,4870.0,4,53,nenhum,03/01/2018,masculino solteiro,carro novo


### Data Treatment

In [198]:
X.isnull().sum()

SALDO_ATUAL         7
RESIDENCIADESDE     0
IDADE               0
OUTROSPLANOSPGTO    0
DATA                0
ESTADOCIVIL         8
PROPOSITO           0
dtype: int64

#### NA Values

In [199]:
mediana = X["SALDO_ATUAL"].median()
mediana

2323.0

In [200]:
# Substituindo os valores nulos pela mediana
# Essa assinatura "{"SALDO_ATUAL": mediana}" é recomendada pelo Pandas para substituir valores nulos usando inplace
# For example, when doing 'df[col].method(value, inplace=True)', try using:
#   'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.
X.fillna({"SALDO_ATUAL": mediana}, inplace=True)

X.isnull().sum()

SALDO_ATUAL         0
RESIDENCIADESDE     0
IDADE               0
OUTROSPLANOSPGTO    0
DATA                0
ESTADOCIVIL         8
PROPOSITO           0
dtype: int64

In [201]:
agrupado = X.groupby(["ESTADOCIVIL"]).size()

agrupado

ESTADOCIVIL
fem div/cas               308
masculino casado/viuvo     92
masculino div/sep          50
masculino solteiro        542
dtype: int64

In [202]:
X.fillna({"ESTADOCIVIL": "masculino solteiro"}, inplace=True)

X.isnull().sum()

SALDO_ATUAL         0
RESIDENCIADESDE     0
IDADE               0
OUTROSPLANOSPGTO    0
DATA                0
ESTADOCIVIL         0
PROPOSITO           0
dtype: int64

#### Outliers

In [203]:
# StandardDeviation - Desvio Padrão
desv = X["SALDO_ATUAL"].std()
desv

685936688.9820064

In [204]:
X.loc[X["SALDO_ATUAL"] >= 2 * desv, "SALDO_ATUAL"]

127    2.541111e+09
160    2.154441e+10
Name: SALDO_ATUAL, dtype: float64

In [205]:
mediana = X["SALDO_ATUAL"].median()
mediana

2323.0

In [206]:
X.loc[X["SALDO_ATUAL"] >= 2 * desv, "SALDO_ATUAL"] = mediana

X.loc[X["SALDO_ATUAL"] >= 2 * desv]

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO


#### Data binding

In [207]:
agrupado = X.groupby(["PROPOSITO"]).size()

agrupado

PROPOSITO
Eletrodomésticos        12
carro novo             234
carro usado            103
educação                50
mobilia/equipamento    181
negócios                97
obras                   22
outros                  12
qualificação             9
radio/tv               280
dtype: int64

In [208]:
X.loc[X["PROPOSITO"] == "Eletrodomésticos", "PROPOSITO"] = "outros"
X.loc[X["PROPOSITO"] == "mobilia/equipamento", "PROPOSITO"] = "outros"
X.loc[X["PROPOSITO"] == "Educação", "PROPOSITO"] = "outros"

agrupado = X.groupby(["PROPOSITO"]).size()

agrupado

PROPOSITO
carro novo      234
carro usado     103
educação         50
negócios         97
obras            22
outros          205
qualificação      9
radio/tv        280
dtype: int64

In [209]:
X["DATA"] = pd.to_datetime(X["DATA"], format="%d/%m/%Y")

In [210]:
X["DATA"].head()

0   2019-01-01
1   2020-01-01
2   2020-01-02
3   2019-01-02
4   2018-01-03
Name: DATA, dtype: datetime64[ns]

In [211]:
X["ANO"] = X["DATA"].dt.year
X["MES"] = X["DATA"].dt.month
X["DIADASEMANA"] = X["DATA"].dt.day_name()

In [212]:
X["DIADASEMANA"].head()

0      Tuesday
1    Wednesday
2     Thursday
3    Wednesday
4    Wednesday
Name: DIADASEMANA, dtype: object

In [213]:
X["ESTADOCIVIL"].unique()

array(['masculino solteiro', 'fem div/cas', 'masculino div/sep',
       'masculino casado/viuvo'], dtype=object)

In [214]:
X["PROPOSITO"].unique()

array(['radio/tv', 'educação', 'outros', 'carro novo', 'carro usado',
       'negócios', 'obras', 'qualificação'], dtype=object)

In [215]:
X["DIADASEMANA"].unique()

array(['Tuesday', 'Wednesday', 'Thursday', 'Saturday', 'Sunday', 'Monday',
       'Friday'], dtype=object)

#### Label Encoder

In [216]:
labelEncoder1 = LabelEncoder()

X["ESTADOCIVIL"] = labelEncoder1.fit_transform(X["ESTADOCIVIL"])
X["PROPOSITO"] = labelEncoder1.fit_transform(X["PROPOSITO"])
X["DIADASEMANA"] = labelEncoder1.fit_transform(X["DIADASEMANA"])

In [217]:
X.head()

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,ANO,MES,DIADASEMANA
0,1169.0,4,67,nenhum,2019-01-01,3,7,2019,1,5
1,5951.0,2,22,nenhum,2020-01-01,0,7,2020,1,6
2,2096.0,3,49,nenhum,2020-01-02,3,2,2020,1,4
3,7882.0,4,45,nenhum,2019-01-02,3,5,2019,1,6
4,4870.0,4,53,nenhum,2018-01-03,3,0,2018,1,6


#### One-hote Encoder

In [218]:
outros = X["OUTROSPLANOSPGTO"].unique()

outros

array(['nenhum', 'banco', 'stores'], dtype=object)

In [219]:
z = pd.get_dummies(X["OUTROSPLANOSPGTO"], prefix="OUTROS", dtype=int)

z.head()

,OUTROS_banco,OUTROS_nenhum,OUTROS_stores
0,0,1,0
1,0,1,0
2,0,1,0
3,0,1,0
4,0,1,0


##### Z-score standardization

In [220]:
sc = StandardScaler()

m = sc.fit_transform(X.iloc[:, 0:3])

m[:10]

array([[-0.74551643,  1.04698668,  1.6392759 ],
       [ 0.95774038, -0.76597727, -0.74024139],
       [-0.41533679,  0.14050471,  0.68746898],
       [ 1.6455256 ,  1.04698668,  0.47595634],
       [ 0.57270888,  1.04698668,  0.89898163],
       [ 2.06332573,  1.04698668, -0.05282528],
       [-0.15211914,  1.04698668,  0.89898163],
       [ 1.31285269, -0.76597727, -0.05282528],
       [-0.07233463,  1.04698668,  1.32200693],
       [ 0.70235871, -0.76597727, -0.42297242]])

In [221]:
X = pd.concat(
    [X, z, pd.DataFrame(m, columns=["SALDO_ATUAL_N", "RESIDENCIADESDE_N", "IDADE_N"])],
    axis=1,
)

X.head()

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,ANO,MES,DIADASEMANA,OUTROS_banco,OUTROS_nenhum,OUTROS_stores,SALDO_ATUAL_N,RESIDENCIADESDE_N,IDADE_N
0,1169.0,4,67,nenhum,2019-01-01,3,7,2019,1,5,0,1,0,-0.745516,1.046987,1.639276
1,5951.0,2,22,nenhum,2020-01-01,0,7,2020,1,6,0,1,0,0.957740,-0.765977,-0.740241
2,2096.0,3,49,nenhum,2020-01-02,3,2,2020,1,4,0,1,0,-0.415337,0.140505,0.687469
3,7882.0,4,45,nenhum,2019-01-02,3,5,2019,1,6,0,1,0,1.645526,1.046987,0.475956
4,4870.0,4,53,nenhum,2018-01-03,3,0,2018,1,6,0,1,0,0.572709,1.046987,0.898982


#### Drop Columns

In [222]:
# Excluindo as colunas que foram transformadas/normalizadas
X.drop(
    columns=[
        "SALDO_ATUAL",
        "RESIDENCIADESDE",
        "IDADE",
        "OUTROSPLANOSPGTO",
        "DATA",
        "OUTROS_banco",
    ],
    inplace=True,
)

X.head()

,ESTADOCIVIL,PROPOSITO,ANO,MES,DIADASEMANA,OUTROS_nenhum,OUTROS_stores,SALDO_ATUAL_N,RESIDENCIADESDE_N,IDADE_N
0,3,7,2019,1,5,1,0,-0.745516,1.046987,1.639276
1,0,7,2020,1,6,1,0,0.957740,-0.765977,-0.740241
2,3,2,2020,1,4,1,0,-0.415337,0.140505,0.687469
3,3,5,2019,1,6,1,0,1.645526,1.046987,0.475956
4,3,0,2018,1,6,1,0,0.572709,1.046987,0.898982


### Random Forest

In [223]:
# Divisão dos dados em treinamento e teste
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(
    X, y, test_size=0.3, random_state=0
)

In [224]:
floresta = RandomForestClassifier(n_estimators=1000)
floresta.fit(X_treinamento, y_treinamento)

RandomForestClassifier(n_estimators=1000)

In [225]:
previsao = floresta.predict(X_teste)

confusao = confusion_matrix(y_teste, previsao)

taxa_acerto = accuracy_score(y_teste, previsao)

taxa_acerto

0.7133333333333334